# Traffic Accident Data Exploration and Cleanup

Our primary goal was to identify the US states that would most benefit from additional spending on traffic safety.

## Primary API

Researching possible APIs to use, we found only one that provided historical data on traffic accidents, here.com. Experimentation with the API provided no results. Searching their Stack Overlow support questions revealed that contrary to documentation, time-based queries were not supported, however it was possible to get historical data through an official support ticket. A request was made, but not answered, likely due to our use of the freemium pricing plan.

We did investigate other options, most notably the National Highway Traffic Safety Administration's FARS database. FARS stands for Fatality Analysis Reporting System. There is a tremendous amount of information to be found on their various sites and in their APIs, but most of it is not easily accessible given our current tools and our limited time.

### API Capabilities

The here.com API had several options for querying data on active accidents, the most expedient being to provide a latitude and longitude, along with a radius not to exceed 100 km. We were limited to 250,000 such request per month. We decided that if we focused on the most populous cities in the US, we would be able to collect enough data in a week to meet our needs.

This led us to create a data collection script to poll for active accidents in 322 cities, which was then automated to run every 9 minutes, minus some downtime when the PC running it was being used for other things. The final query count for our data was a little under 180,000. If the results included a list of vehicular accidents, we saved it out as a JSON file. The final count of uniquely identified accidents among those files was 6,149. More on that disparity below.

### Population-Based Queries

Since it would not be possible to cover all populations and we were looking to evaluate states against each other, we decided to focus on high-population cities. We found a list on Wikipedia of all US cities with a population greater than 100,000, then added a few additional cities for the five states that lacked them.  Those five states are: Delaware, Maine, Vermont, West Virginia and Wyoming. The following is a map showing the areas being sampled.

![Accident Data Collection Areas](Resources/accident_data_collection.png)

## Data Collation

This process of sampling live data on a timed interval led to a great many duplicate entries, for a variety of reasons:

    1. Accidents typically last longer than 9 minutes
    2. Our polling locations and radii overlapped
    3. Accidents are reported from a variety of sources

Since our data was spread out among tens of thousands of files, and we needed to filter out duplicates, another script was written to collate what we had collected into a single JSON file.

### 1st-Pass Data Cleanup

That collated data file was still very large and unwieldy, so another script was written to extract the usesul data into a Pandas DataFrame and then exported to JSON. However the raw data only provided the lat/lng coordinates for an accident's location. So once we were done fully collecting data, this cleanup script was modified to also query the Google Geocoding API, allowing us to reverse geocode our lat/lng coordinates to identify which US state (if any) an accident occurred in.

## Exploring the Data

Now that the data was easy to interact with, we began exploring it to look for the answers to our questions. But first...

### Verification with GMaps

As noted, collecting the full dataset required about a week of time, but we ran the collation script multiple times throughout the process. After the first time running it, we used the gmaps module to make a heatmap that showed accident locations. This was done to verify that our data made sense. In fact, while peer-reviewing the code and talking through it as part of this exercise, the team realized that a bad conversion of meters to kilometers had occurred, requiring us to fix the script and restart the data collection.

### Building the Final DataFrame

Our next steps as a team overlap "exploring the data" and "analyzing the data", but this document will focus mainly on its exploration and cleanup.

#### Grouping By State

Having weeded out duplicates entries, non-US entries, and irrelevant data, we could finally start asking questions of the data. To begin, we grouped accidents by state, and simply counting how many accidents occured in each.

![Total Accidents by State](Charts/total_accidents_by_state.png)

This looks like there's a problem with our data, but we believe that it's an artifact of how accidents get reported to here.com. To understand why, we need to break out accidents by their Criticality rating.

#### Criticality

Each incident report has a Criticality associated with it. In our first pass cleanup, we collected that info in text format:

1. LowImpact
2. Minor
3. Major
4. Critical

So we had to convert it back to numbers:

1. LowImpact = 3
2. Minor = 2
3. Major = 1
4. Critical = 0

So a smaller number is more severe. LowImpact is only used with non-accident incidents that we were not tracking. We were curious what the chart above would look like if we grouped Major and Critical accidents into a Severe category that ignored Minor accidents, and got the following. The states with the most accidents are in red.

![Severe Accidents By State](Charts/severe_accidents_by_state.png)

Sorting that by count reveals our top contenders.

![Severe Accidents by State Sorted](Charts/severe_accidents_by_state_sorted.png)

Next we wanted to see if anything changed when we broke out Critical from Major accidents, which gave us the following two charts:

![Breakdown of Severe Accidents by State](Charts/breakdown_of_severe_accidents_by_state.png)

![Sorted Breakdown of Severe Accidents by State](Charts/breakdown_of_severe_accidents_by_state_sorted.png)

Given some of the disparities that jump, we decided to sort by Critical accidents first, then Major ones, which gives us our last chart in this series.

![Breakdown of Severe Accidents by State Sorted by Most Critical](Charts/breakdown_of_severe_accidents_by_state_sorted_by_most_critical.png)

Our final exploration of the data was to chart number of accidents throughout the day by hour, highlighting the times with the most accidents in red.

![Total Accidents by Hour](Charts/total_accidents_by_hour.png)